<a href="https://colab.research.google.com/github/seanard1/NHL_Fantasy/blob/main/Sked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz
import scipy.stats

In [4]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/schedule'

start = '?startDate=2023-10-23'

end = '&endDate=2023-10-25'

In [5]:
skedURL = baseURL + start + end

sked = requests.get(skedURL).json()

In [6]:
days = sked['dates']

eastern = pytz.timezone('US/Eastern')

In [7]:
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams'

teams = requests.get(teamURL).json()
teams = teams['teams']

team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['abbreviation']
    name = team['name']
    id = team['id']

    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name], 'id': [id]})

    team_names = pd.concat([team_names, temp], ignore_index=True)

In [8]:
# See teams with most games

teams_data = {}
games_data = []

for day in days:
    games = day['games']

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']

        if home_team not in teams_data:
            teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
        if away_team not in teams_data:
            teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

        teams_data[home_team]['played'] += 1
        teams_data[away_team]['played'] += 1
        teams_data[home_team]['home'] += 1
        teams_data[away_team]['away'] += 1

        games_data.append(game['gamePk'])

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

teams_df

,played,home,away
Buffalo Sabres,2,1,1
Montréal Canadiens,2,1,1
Washington Capitals,2,1,1
New Jersey Devils,2,1,1
Columbus Blue Jackets,1,1,0
Ottawa Senators,1,1,0
Tampa Bay Lightning,1,1,0
Pittsburgh Penguins,1,1,0
Florida Panthers,1,1,0
New York Islanders,1,1,0


In [9]:
# See teams that play back-to-back

def check_consecutive_dates(game1, game2):
    date_format = '%Y-%m-%dT%H:%M:%SZ'
    date1 = datetime.strptime(game1['gameDate'], date_format)
    date2 = datetime.strptime(game2['gameDate'], date_format)
    return (date2 - date1).days == 1

teams_playing_consecutive_days = set()

for i in range(len(days)-1):
    current_day_games = days[i]['games']
    next_day_games = days[i+1]['games']

    teams_on_next_day = set()

    for game2 in next_day_games:
        home_team_2 = game2['teams']['home']['team']['name']
        away_team_2 = game2['teams']['away']['team']['name']
        teams_on_next_day.add(home_team_2)
        teams_on_next_day.add(away_team_2)

    for game1 in current_day_games:
        home_team_1 = game1['teams']['home']['team']['name']
        away_team_1 = game1['teams']['away']['team']['name']

        if (home_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(home_team_1)
        elif (away_team_1 in teams_on_next_day) and check_consecutive_dates(game1, game2):
            teams_playing_consecutive_days.add(away_team_1)

print("Teams playing on consecutive days:")
for team in teams_playing_consecutive_days:
    print(team)

Teams playing on consecutive days:
Buffalo Sabres
Washington Capitals
New Jersey Devils


In [10]:
str(games_data[0]-1)[-2:]

'81'

In [11]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202302'
game = 1
range_end = int(str(games_data[0]-1)[-2:])

awaylogs = {}
homelogs = {}


for i in range(range_end-2):

    gameURL = baseURL + year + str(i+1).zfill(4) + appendix

    print(gameURL)

    game = requests.get(gameURL).json()

    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = away[player]['stats']
        try:
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        birth_date = birthday[player]['birthDate']
        stats = home[player]['stats']
        try:
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player

#         game+=1

https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020016/feed/live
https://

In [12]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)


away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()


away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_df = pd.concat([home_df, away_df], axis=0)

all_df['date'] = pd.to_datetime(all_df['date'], format='%d-%m-%Y')
all_df['birthdate'] = pd.to_datetime(all_df['birthdate'], format='%Y-%m-%d')
player_list = all_df['name'].unique()
all_df['name'].nunique()
all_df = all_df.sort_values(by='date')

In [13]:
pd.set_option('display.max_columns', None)
all_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8479458-10-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-10,Nikita Zaitsev,CHI,PIT,1991-10-29,ID8479458,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8470621-10-10-2023,1008,2,0,2,1,0,0,0,NaN,0,0,0,0,0,0,0,2,707,301,0,RW,2023-10-10,Corey Perry,CHI,PIT,1985-05-16,ID8470621,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8475852-10-10-2023,3600,0,0,41,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Petr Mrazek,CHI,PIT,1992-02-14,ID8475852,0,39,4,0,35,0,37,4,W,95.121951,100.0,NaN,94.594595,0
ID8475797-10-10-2023,838,0,0,0,3,0,0,0,NaN,0,0,0,0,0,0,1,0,790,0,48,D,2023-10-10,Jarred Tinordi,CHI,PIT,1992-02-20,ID8475797,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476897-21-10-2023,813,1,0,1,1,0,0,0,37.5,3,8,0,0,0,0,2,1,743,0,70,C,2023-10-21,Oskar Sundqvist,STL,PIT,1994-03-23,ID8476897,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8475753-21-10-2023,1430,1,0,0,0,0,0,0,NaN,0,0,1,0,0,0,2,3,1257,28,145,D,2023-10-21,Justin Faulk,STL,PIT,1992-03-20,ID8475753,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8477953-21-10-2023,1119,2,0,2,3,0,0,0,0.0,0,1,1,0,0,0,0,1,771,212,136,RW,2023-10-21,Kasperi Kapanen,STL,PIT,1996-07-23,ID8477953,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0
ID8477810-21-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-21,Joel Hanley,DAL,PHI,1991-06-08,ID8477810,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0


In [14]:
all_df.loc[all_df['name'] == 'Devon Levi']

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout
ID8482221-12-10-2023,3534,0,0,30,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-12,Devon Levi,BUF,NYR,2001-12-27,ID8482221,0,26,3,0,23,1,25,4,L,86.666667,75.0,0.0,92.0,0
ID8482221-14-10-2023,3572,0,0,29,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-14,Devon Levi,BUF,NYI,2001-12-27,ID8482221,0,26,3,0,23,0,26,3,L,89.655172,100.0,NaN,88.461538,0
ID8482221-17-10-2023,3706,0,0,25,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-17,Devon Levi,BUF,TBL,2001-12-27,ID8482221,0,23,3,0,20,0,22,3,W,92.0,100.0,NaN,90.909091,0
ID8482221-19-10-2023,3538,0,0,36,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-19,Devon Levi,BUF,CGY,2001-12-27,ID8482221,0,32,8,1,23,1,27,8,L,88.888889,100.0,100.0,85.185185,0
ID8482221-21-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-21,Devon Levi,BUF,NYI,2001-12-27,ID8482221,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0


In [15]:
for index, row in all_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
      all_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'decisionPoints'] = d
        all_df.at[index, 'wins'] = w
#         row['fantasyPoints'] = fp
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_df.at[index, 'fantasyPoints'] = fp
        all_df.at[index, 'specialTeams'] = st
#         row['fantasyPoints'] = fp

In [16]:
all_df



,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,15,1,0,0,0,0,-1,690,87,225,C,2023-10-10,Anthony Cirelli,TBL,NSH,1997-07-15,ID8478519,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8479458-10-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-10,Nikita Zaitsev,CHI,PIT,1991-10-29,ID8479458,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
ID8470621-10-10-2023,1008,2,0,2,1,0,0,0,NaN,0,0,0,0,0,0,0,2,707,301,0,RW,2023-10-10,Corey Perry,CHI,PIT,1985-05-16,ID8470621,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.3,0.0,NaN,NaN,NaN,NaN
ID8475852-10-10-2023,3600,0,0,41,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,G,2023-10-10,Petr Mrazek,CHI,PIT,1992-02-14,ID8475852,0,39,4,0,35,0,37,4,W,95.121951,100.0,NaN,94.594595,0,1.0,7.8,NaN,41.0,2.0,4.0,1.0
ID8475797-10-10-2023,838,0,0,0,3,0,0,0,NaN,0,0,0,0,0,0,1,0,790,0,48,D,2023-10-10,Jarred Tinordi,CHI,PIT,1992-02-20,ID8475797,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.8,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476897-21-10-2023,813,1,0,1,1,0,0,0,37.5,3,8,0,0,0,0,2,1,743,0,70,C,2023-10-21,Oskar Sundqvist,STL,PIT,1994-03-23,ID8476897,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
ID8475753-21-10-2023,1430,1,0,0,0,0,0,0,NaN,0,0,1,0,0,0,2,3,1257,28,145,D,2023-10-21,Justin Faulk,STL,PIT,1992-03-20,ID8475753,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.0,0.0,NaN,NaN,NaN,NaN
ID8477953-21-10-2023,1119,2,0,2,3,0,0,0,0.0,0,1,1,0,0,0,0,1,771,212,136,RW,2023-10-21,Kasperi Kapanen,STL,PIT,1996-07-23,ID8477953,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.5,0.0,NaN,NaN,NaN,NaN
ID8477810-21-10-2023,0,0,0,0,0,0,0,0,NaN,0,0,0,0,0,0,0,0,0,0,0,N/A,2023-10-21,Joel Hanley,DAL,PHI,1991-06-08,ID8477810,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [17]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats = all_df.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats['gamePlayed'] = summary_stats['gamePlayed'].astype(int)
summary_stats['decisionPoints'] = summary_stats['decisionPoints'].astype(int)
summary_stats['wins'] = summary_stats['wins'].astype(int)
summary_stats['specialTeams'] = summary_stats['specialTeams'].astype(int)


summary_stats['FPP60'] = (summary_stats['fantasyPoints'] / summary_stats['timeOnIce'] * 3600).round(2)
summary_stats['FPPG'] = (summary_stats['fantasyPoints'] / summary_stats['gamePlayed']).round(2)

summary_stats = summary_stats.sort_values(by='FPPG', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats.set_index('name', inplace=True)

summary_stats.head()

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,3600,1,0,0,0,23,0,0,0,0,0,23,6,0,17,0,17,6,4,1,1,11.6,11.60,11.60
Alex Nedeljkovic,ID8477968,PIT,G,3582,1,0,0,0,36,0,0,0,0,0,34,3,0,31,0,32,4,4,1,0,6.8,6.83,6.80
Eric Comrie,ID8477480,BUF,G,3600,1,0,0,0,25,0,0,0,0,0,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80
Alexandar Georgiev,ID8480382,COL,G,18252,5,0,0,0,140,0,0,0,0,0,132,29,7,96,7,103,30,20,5,1,33.4,6.59,6.68
Jeremy Swayman,ID8480280,BOS,G,7141,2,0,0,0,70,0,0,0,0,0,66,21,4,41,4,44,22,8,2,0,13.2,6.65,6.60


In [18]:
summary_stats.loc[summary_stats['team'] == 'BUF']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Eric Comrie,ID8477480,BUF,G,3600,1,0,0,0,25,0,0,0,0,0,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80
Rasmus Dahlin,ID8480839,BUF,D,7324,5,0,5,1,14,9,9,6254,927,143,0,0,0,0,0,0,0,0,0,0,12.3,6.05,2.46
Dylan Cozens,ID8481528,BUF,C,4761,5,2,2,1,12,7,0,3686,995,80,0,0,0,0,0,0,0,0,0,0,8.4,6.35,1.68
Erik Johnson,ID8473446,BUF,D,4381,5,1,0,0,7,8,9,3287,0,1094,0,0,0,0,0,0,0,0,0,0,8.0,6.57,1.60
Mattias Samuelsson,ID8480807,BUF,D,5932,5,1,1,0,5,10,7,4898,9,1025,0,0,0,0,0,0,0,0,0,0,8.0,4.86,1.60
Owen Power,ID8482671,BUF,D,7238,5,0,2,0,7,0,10,5967,555,716,0,0,0,0,0,0,0,0,0,0,7.7,3.83,1.54
Jeff Skinner,ID8475784,BUF,LW,5192,5,2,1,0,16,0,2,4163,988,41,0,0,0,0,0,0,0,0,0,0,7.6,5.27,1.52
Casey Mittelstadt,ID8479999,BUF,C,4998,5,1,3,0,6,0,2,4419,571,8,0,0,0,0,0,0,0,0,0,0,6.6,4.75,1.32
Jordan Greenway,ID8478413,BUF,LW,4855,5,1,2,0,4,8,2,3813,5,1037,0,0,0,0,0,0,0,0,0,0,6.2,4.60,1.24


In [19]:
# https://statsapi.web.nhl.com/api/v1/people/8477953/stats?stats=gameLog&season=20232024
# This is a game log endpoint


skaters = all_df.loc[all_df['position'] != 'G']

In [20]:
opponent_stats = skaters.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique'
    # Add more columns as needed
})

opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['date'], 2)

opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=True, method='min').astype(int)

opponent_stats = opponent_stats.rename_axis('abbreviation')

opponent_stats = pd.merge(opponent_stats, team_names, on='abbreviation')


In [21]:
skaters_temp = skaters
# replace_dict = {'LW': 'W', 'RW': 'W'}
skaters_temp['position'] = skaters_temp['position'].replace('LW', 'W')
skaters_temp['position'] = skaters_temp['position'].replace('RW', 'W')
skaters_temp = skaters_temp.loc[skaters_temp['gamePlayed']>0]


opponent_stats_by_pos = skaters_temp.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))]
    # Add more columns as needed
})
opponent_stats_by_pos

<ipython-input-21-03b2cb2a1286>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skaters_temp['position'] = skaters_temp['position'].replace('LW', 'W')
<ipython-input-21-03b2cb2a1286>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skaters_temp['position'] = skaters_temp['position'].replace('RW', 'W')


timeOnIce assists goals shots hits blocked fantasyPoints  \
                        sum     sum   sum   sum  sum     sum           sum   
opponent position                                                            
ANA      C            25726       9     5    46   20      12          35.6   
         D            27362      12     2    42   24      31          42.1   
         W            17189       3     5    36   19      15          28.5   
ARI      C            26399       6     2    55   39      16          30.4   
         D            35018       4     1    46   30      41          35.6   
...                     ...     ...   ...   ...  ...     ...           ...   
WPG      D            34399       7     5    46   35      51          53.1   
         W            19985       9     6    44   25      16          37.9   
WSH      C            21687      12     8    46   16      11          44.7   
         D            27682       7     2    38   15      44          38.8   
         W            22126       9     5    48   19      12          33.7   

                         
                   mean  
opponent position        
ANA      C         1.32  
         D         1.75  
         W         1.36  
ARI      C         1.09  
         D         1.19  
...                 ...  
WPG      D         1.71  
         W         1.58  
WSH      C         1.94  
         D         1.62  
         W         1.35  

[96 rows x 8 columns]

In [22]:
top_players_by_team = summary_stats.groupby('team').apply(lambda x: x.nlargest(5, 'FPPG'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team.loc[top_players_by_team['team'] == 'ANA']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Frank Vatrano,ID8478366,ANA,RW,4605,4,4,0,1,14,9,8,3404,431,770,0,0,0,0,0,0,0,0,0,0,14.8,11.57,3.70
Lukas Dostal,ID8480843,ANA,G,7051,2,0,0,0,62,0,0,0,0,0,57,15,0,42,0,44,18,4,1,0,5.4,2.76,2.70
Cam Fowler,ID8475764,ANA,D,5534,4,1,1,0,4,1,8,4148,472,914,0,0,0,0,0,0,0,0,0,0,7.5,4.88,1.88
Pavel Mintyukov,ID8483490,ANA,D,4710,4,1,1,0,7,8,5,3756,766,188,0,0,0,0,0,0,0,0,0,0,7.0,5.35,1.75
Ilya Lyubushkin,ID8480950,ANA,D,3905,4,0,0,0,2,10,11,3357,0,548,0,0,0,0,0,0,0,0,0,0,6.7,6.18,1.67


In [23]:
eastern = pytz.timezone('US/Eastern')

for day in days:
    date = day['date']
    games = day['games']

    day_of_week = datetime.strptime(date, '%Y-%m-%d').strftime('%A').upper()
    print(day_of_week)

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']
        venue = game['venue']['name']
        game_time_utc = datetime.fromisoformat(game['gameDate'][:-1]).replace(tzinfo=pytz.utc)
        game_time_et = game_time_utc.astimezone(eastern)

        formatted_time = game_time_et.strftime('%I:%M %p').lstrip('0').replace(':00', '').replace('PM', 'p.m.')

        home_abbr = opponent_stats.loc[opponent_stats['name'] == home_team, 'abbreviation'].values[0]
        home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]

        away_abbr = opponent_stats.loc[opponent_stats['name'] == away_team, 'abbreviation'].values[0]
        away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]

#         print(f"On {date}, {away_team} at {home_team}")
        print(f"\033[1m{away_team} at {home_team}\033[0m")
        print(f"{formatted_time}, {venue}")
        print()
        print(f"{away_abbr}: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPPG\033[0m")
        for index, row in away_top5.iterrows():
            print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")
        print()
        print(f"{home_abbr}: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {opponent_stats.loc[opponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
        print(f"\033[1mTop Players in FPPG\033[0m")
        for index, row in home_top5.iterrows():
            print(f"{index}, FPPG: {row['FPPG']}, FP: {round(row['fantasyPoints'], 2)}")

        print()
#         print("\033[1mThis is bold text\033[0m")


MONDAY
Montréal Canadiens at Buffalo Sabres
7 p.m., KeyBank Center

MTL: 27.72 FPAPG, Rank: 23
Top Players in FPPG
Cole Caufield, FPPG: 2.8, FP: 11.2
Mike Matheson, FPPG: 2.72, FP: 10.9
Jake Allen, FPPG: 2.3, FP: 4.6
Tanner Pearson, FPPG: 2.1, FP: 8.4
Kaiden Guhle, FPPG: 2.07, FP: 6.2

BUF: 25.8 FPAPG, Rank: 17
Top Players in FPPG
Eric Comrie, FPPG: 6.8, FP: 6.8
Rasmus Dahlin, FPPG: 2.46, FP: 12.3
Dylan Cozens, FPPG: 1.68, FP: 8.4
Erik Johnson, FPPG: 1.6, FP: 8.0
Mattias Samuelsson, FPPG: 1.6, FP: 8.0

TUESDAY
Toronto Maple Leafs at Washington Capitals
6 p.m., Capital One Arena

TOR: 30.16 FPAPG, Rank: 30
Top Players in FPPG
Joseph Woll, FPPG: 6.1, FP: 12.2
Auston Matthews, FPPG: 4.14, FP: 20.7
William Nylander, FPPG: 3.5, FP: 17.5
John Tavares, FPPG: 3.48, FP: 17.4
TJ Brodie, FPPG: 2.36, FP: 11.8

WSH: 29.3 FPAPG, Rank: 27
Top Players in FPPG
John Carlson, FPPG: 2.88, FP: 11.5
Nic Dowd, FPPG: 1.75, FP: 3.5
Martin Fehervary, FPPG: 1.45, FP: 5.8
Dylan Strome, FPPG: 1.38, FP: 5.5
Matthew

In [24]:
team_check = opponent_stats['name'].values

teams_df.index.values

for team in team_check:
    if team not in teams_df.index.values:
        print(team)

In [25]:
summary_stats = summary_stats.dropna()

In [26]:
# summary_stats = summary_stats[summary_stats['position'] != 'N/A']
summary_stats

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,3600,1,0,0,0,23,0,0,0,0,0,23,6,0,17,0,17,6,4,1,1,11.6,11.60,11.60
Alex Nedeljkovic,ID8477968,PIT,G,3582,1,0,0,0,36,0,0,0,0,0,34,3,0,31,0,32,4,4,1,0,6.8,6.83,6.80
Eric Comrie,ID8477480,BUF,G,3600,1,0,0,0,25,0,0,0,0,0,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80
Alexandar Georgiev,ID8480382,COL,G,18252,5,0,0,0,140,0,0,0,0,0,132,29,7,96,7,103,30,20,5,1,33.4,6.59,6.68
Jeremy Swayman,ID8480280,BOS,G,7141,2,0,0,0,70,0,0,0,0,0,66,21,4,41,4,44,22,8,2,0,13.2,6.65,6.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stuart Skinner,ID8479973,EDM,G,9232,3,0,0,0,59,0,0,0,0,0,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80
Pyotr Kochetkov,ID8481611,CAR,G,5617,2,0,0,0,45,0,0,0,0,0,37,5,1,31,2,35,8,0,0,0,-8.6,-5.51,-4.30
Joel Hofer,ID8480981,STL,G,3589,1,0,0,0,42,0,0,0,0,0,36,3,2,31,2,34,6,0,0,0,-4.8,-4.81,-4.80


In [27]:
skaters_summary = summary_stats.loc[summary_stats['position'] != "G"]
skaters_summary = skaters_summary.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])
# skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']


In [28]:
goalies_summary = summary_stats.loc[summary_stats['position'] == "G"]
goalies_summary = goalies_summary.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])
goalies_summary

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,3600,1,23,23,6,0,17,0,17,6,4,1,1,11.6,11.60,11.60
Alex Nedeljkovic,ID8477968,PIT,G,3582,1,36,34,3,0,31,0,32,4,4,1,0,6.8,6.83,6.80
Eric Comrie,ID8477480,BUF,G,3600,1,25,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80
Alexandar Georgiev,ID8480382,COL,G,18252,5,140,132,29,7,96,7,103,30,20,5,1,33.4,6.59,6.68
Jeremy Swayman,ID8480280,BOS,G,7141,2,70,66,21,4,41,4,44,22,8,2,0,13.2,6.65,6.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stuart Skinner,ID8479973,EDM,G,9232,3,59,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80
Pyotr Kochetkov,ID8481611,CAR,G,5617,2,45,37,5,1,31,2,35,8,0,0,0,-8.6,-5.51,-4.30
Joel Hofer,ID8480981,STL,G,3589,1,42,36,3,2,31,2,34,6,0,0,0,-4.8,-4.81,-4.80


In [29]:
team_names
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams/'
appendix = '/stats'

team_stats = pd.DataFrame()

for index, row in team_names.iterrows():

  fetchURL = teamURL + str(row['id']) + appendix

  # print(fetchURL)

  team = requests.get(fetchURL).json()
  stats = team['stats'][0]['splits'][0]['stat']
  name = team['stats'][0]['splits'][0]['team']['name']

  temp = pd.DataFrame([stats])
  temp['name'] = name

  team_stats = pd.concat([team_stats, temp], ignore_index=True)


team_stats = pd.merge(team_stats, team_names, on='name')
team_stats.set_index('name', inplace=True)

In [30]:

team_stats.columns.to_list()


['gamesPlayed',
 'wins',
 'losses',
 'ot',
 'pts',
 'ptPctg',
 'goalsPerGame',
 'goalsAgainstPerGame',
 'evGGARatio',
 'powerPlayPercentage',
 'powerPlayGoals',
 'powerPlayGoalsAgainst',
 'powerPlayOpportunities',
 'penaltyKillPercentage',
 'shotsPerGame',
 'shotsAllowed',
 'winScoreFirst',
 'winOppScoreFirst',
 'winLeadFirstPer',
 'winLeadSecondPer',
 'winOutshootOpp',
 'winOutshotByOpp',
 'faceOffsTaken',
 'faceOffsWon',
 'faceOffsLost',
 'faceOffWinPercentage',
 'shootingPctg',
 'savePctg',
 'abbreviation',
 'id']

In [33]:
for index, row in skaters_summary.iterrows():
  team = row['team']
  gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

  skaters_summary.loc[index, 'gamesRemain'] = int(gr)

  skaters_summary.loc[index, 'fantasyPace'] = row['FPPG'] * int(gr) + int(row['fantasyPoints'])


skaters_summary['gamesRemain'] = skaters_summary['gamesRemain'].astype(int)

In [39]:
skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']

goodAho = skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR')]
goodAhoGR = 82 - team_stats.loc[team_stats['abbreviation'] == 'CAR']['gamesPlayed']
goodAhoPace = goodAho['FPPG'] * int(gr) + int(goodAho['fantasyPoints'])

skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR'), 'fantasyPace'] = int(goodAhoPace)
skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'CAR'), 'gamesRemain'] = int(goodAhoGR)

badAho = skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI')]
badAhoGR = 82 - team_stats.loc[team_stats['abbreviation'] == 'NYI']['gamesPlayed']
badAhoPace = badAho['FPPG'] * int(gr) + int(badAho['fantasyPoints'])

skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI'), 'fantasyPace'] = int(badAhoPace)
skaters_summary.loc[(skaters_summary.index == 'Sebastian Aho') & (skaters_summary['team'] == 'NYI'), 'gamesRemain'] = int(badAhoGR)

In [40]:
skaters_summary.loc[skaters_summary.index == 'Sebastian Aho']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace
name,,,,,,,,,,,,,,,,,,,
Sebastian Aho,ID8478427,CAR,C,3780,3,1,2,3,8,1,0,2396,981,403,6.4,6.10,2.13,76,170.0
Sebastian Aho,ID8480222,NYI,D,3392,4,0,0,0,5,1,6,2878,405,109,3.6,3.82,0.90,78,72.0


In [41]:
for index, row in team_stats.iterrows():
  team_stats.at[index, 'creaseMins'] = goalies_summary.loc[goalies_summary['team'] == row['abbreviation']]['timeOnIce'].sum()

team_stats['creaseMins'] = team_stats['creaseMins'].astype(int)

In [42]:
for index, row in goalies_summary.iterrows():
  team = row['team']
  gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

  goalies_summary.at[index, 'gamesRemain'] = int(gr)
  cs = row['timeOnIce'] / team_stats.loc[team_stats['abbreviation'] == team]['creaseMins'].sum()


  goalies_summary.at[index, 'fantasyPace'] = (float(row['FPP60']) * int(gr) * cs) + int(row['fantasyPoints'])
  goalies_summary.at[index, 'creaseShare'] = cs * 100

# goalies_summary['gamesRemain'] = goalies_summary['gamesRemain'].astype(int)
# goalies_summary['creaseShare'] = round(goalies_summary['creaseShare'].astype(int), 2)

In [43]:
goalies_summary['gamesRemain'] = goalies_summary['gamesRemain'].astype(int)
goalies_summary['fantasyPace'] = round(goalies_summary['fantasyPace'], 2)
goalies_summary['creaseShare'] = round(goalies_summary['creaseShare'], 2)

goalies_summary

,id,team,position,timeOnIce,gamePlayed,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG,gamesRemain,fantasyPace,creaseShare
name,,,,,,,,,,,,,,,,,,,,,,
James Reimer,ID8473503,DET,G,3600,1,23,23,6,0,17,0,17,6,4,1,1,11.6,11.60,11.60,77,191.35,20.19
Alex Nedeljkovic,ID8477968,PIT,G,3582,1,36,34,3,0,31,0,32,4,4,1,0,6.8,6.83,6.80,77,112.56,20.26
Eric Comrie,ID8477480,BUF,G,3600,1,25,24,5,0,19,0,20,5,4,1,0,6.8,6.80,6.80,77,111.01,20.06
Alexandar Georgiev,ID8480382,COL,G,18252,5,140,132,29,7,96,7,103,30,20,5,1,33.4,6.59,6.68,77,540.43,100.00
Jeremy Swayman,ID8480280,BOS,G,7141,2,70,66,21,4,41,4,44,22,8,2,0,13.2,6.65,6.60,78,271.41,49.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stuart Skinner,ID8479973,EDM,G,9232,3,59,48,11,2,35,3,41,15,1,0,0,-11.4,-4.45,-3.80,77,-185.94,51.06
Pyotr Kochetkov,ID8481611,CAR,G,5617,2,45,37,5,1,31,2,35,8,0,0,0,-8.6,-5.51,-4.30,76,-116.82,25.99
Joel Hofer,ID8480981,STL,G,3589,1,42,36,3,2,31,2,34,6,0,0,0,-4.8,-4.81,-4.80,78,-93.89,23.96


In [45]:
last_season = pd.read_csv('2022-23.csv')
season_prior = pd.read_csv('2021-22.csv')
two_years_ago = pd.read_csv('2020-21.csv')


last_season['date'] = pd.to_datetime(last_season['date'])
season_prior['date'] = pd.to_datetime(season_prior['date'])
two_years_ago['date'] = pd.to_datetime(two_years_ago['date'])

last_season['birthdate'] = pd.to_datetime(last_season['birthdate'])
season_prior['birthdate'] = pd.to_datetime(season_prior['birthdate'])
two_years_ago['birthdate'] = pd.to_datetime(two_years_ago['birthdate'])

In [46]:
for index, row in last_season.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      last_season.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        last_season.at[index, 'decisionPoints'] = d
        last_season.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        last_season.at[index, 'specialTeams'] = st

for index, row in season_prior.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      season_prior.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        season_prior.at[index, 'decisionPoints'] = d
        season_prior.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        season_prior.at[index, 'specialTeams'] = st

for index, row in two_years_ago.iterrows():
    w = 0

    if row['timeOnIce'] > 0:
      two_years_ago.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        two_years_ago.at[index, 'decisionPoints'] = d
        two_years_ago.at[index, 'wins'] = w
    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        two_years_ago.at[index, 'specialTeams'] = st

# Bring in previous seasons

In [47]:
last_three = pd.concat([all_df, last_season, season_prior], axis=0)

last_three = last_three.loc[last_three['timeOnIce'] > 0]

In [321]:
# for index, row in last_season.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       last_season.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             last_season.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         last_season.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         last_season.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         last_season.at[index, 'fantasyPoints'] = fp
#         last_season.at[index, 'decisionPoints'] = d
#         last_season.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         last_season.at[index, 'fantasyPoints'] = fp
#         last_season.at[index, 'specialTeams'] = st

In [322]:
# for index, row in season_prior.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       season_prior.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             season_prior.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         season_prior.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         season_prior.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         season_prior.at[index, 'fantasyPoints'] = fp
#         season_prior.at[index, 'decisionPoints'] = d
#         season_prior.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         season_prior.at[index, 'fantasyPoints'] = fp
#         season_prior.at[index, 'specialTeams'] = st

In [323]:
# for index, row in two_years_ago.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       two_years_ago.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             two_years_ago.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         two_years_ago.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         two_years_ago.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         two_years_ago.at[index, 'fantasyPoints'] = fp
#         two_years_ago.at[index, 'decisionPoints'] = d
#         two_years_ago.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         two_years_ago.at[index, 'fantasyPoints'] = fp
#         two_years_ago.at[index, 'specialTeams'] = st

In [324]:
# last_season.to_csv('f2022-23.csv')
# # season_prior.to_csv('f2021-22.csv')
# two_years_ago.to_csv('f2020-21.csv')

In [320]:
# for index, row in last_three.iterrows():
#     d = 0
#     so = 0
#     w = 0

#     if row['timeOnIce'] > 0:
#       last_three.at[index, 'gamePlayed'] = 1


#     if row['position'] == "G":
#         if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
#             last_three.at[index, 'shutout'] = 0
#         if row['shutout'] == 1:
#             so = 3
#         else:
#             so = 0
#         if row['decision'] == "W":
#             d = 4
#             w += 1
#         elif row['decision'] == "OTL":
#             d = 1
#         elif row['decision'] == "":
#             d = 0
#         else:
#             d = 0
#         shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
#         last_three.at[index, 'shots_against'] = shots_against
#         ga = (shots_against - row['saves']) * -2
#         last_three.at[index, 'goals_against'] = abs(ga)/2
#         saves = row['saves'] * .2
#         fp = d + so + ga + saves
#         last_three.at[index, 'fantasyPoints'] = fp
#         last_three.at[index, 'decisionPoints'] = d
#         last_three.at[index, 'wins'] = w
# #         row['fantasyPoints'] = fp
#     else:
#         st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
#         fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
#         last_three.at[index, 'fantasyPoints'] = fp
#         last_three.at[index, 'specialTeams'] = st
# #         row['fantasyPoints'] = fp

In [48]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_three = last_three.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_three.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_three['gamePlayed'] = summary_stats_last_three['gamePlayed'].astype(int)
summary_stats_last_three['decisionPoints'] = summary_stats_last_three['decisionPoints'].astype(int)
summary_stats_last_three['wins'] = summary_stats_last_three['wins'].astype(int)
summary_stats_last_three['specialTeams'] = summary_stats_last_three['specialTeams'].astype(int)


summary_stats_last_three['FPP60'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['timeOnIce'] * 3600).round(2)
summary_stats_last_three['FPPG'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['gamePlayed']).round(2)

summary_stats_last_three = summary_stats_last_three.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_three = summary_stats_last_three[summary_stats_last_three['position'] != 'N/A']

summary_stats_last_three.info()

skaters_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] != "G"]
skaters_summary_last_three = skaters_summary_last_three.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] == "G"]
goalies_summary_last_three = goalies_summary_last_three.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 1299 entries, Juuse Saros to Jett Alexander
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1299 non-null   object 
 1   team                     1299 non-null   object 
 2   position                 1299 non-null   object 
 3   timeOnIce                1299 non-null   int64  
 4   gamePlayed               1299 non-null   int64  
 5   goals                    1299 non-null   int64  
 6   assists                  1299 non-null   int64  
 7   specialTeams             1299 non-null   int64  
 8   shots                    1299 non-null   int64  
 9   hits                     1299 non-null   int64  
 10  blocked                  1299 non-null   int64  
 11  evenTimeOnIce            1299 non-null   int64  
 12  powerPlayTimeOnIce       1299 non-null   int64  
 13  shortHandedTimeOnIce     1299 non-null   int64  
 14  saves    

In [49]:
skaters_summary_last_three

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,
Brent Burns,ID8470613,CAR,D,251015,170,30,89,45,472,97,244,196866,29265,24884,350.4,5.03,2.06
Rasmus Dahlin,ID8480839,BUF,D,243317,163,28,103,55,388,235,232,200290,32481,10546,364.8,5.40,2.24
Quinn Hughes,ID8480800,VAN,D,242738,159,16,133,72,322,49,132,194204,36279,12255,304.1,4.51,1.91
Victor Hedman,ID8475167,TBL,D,240537,164,30,110,57,418,191,277,190748,30772,19017,397.9,5.96,2.43
Adam Fox,ID8479323,NYR,D,238606,165,24,127,71,324,58,258,185504,32383,20719,377.7,5.70,2.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Seth Griffith,ID8476495,EDM,C,337,1,0,0,0,0,1,0,337,0,0,0.1,1.07,0.10
Mike Vecchione,ID8480087,WSH,RW,306,1,0,0,0,3,0,0,306,0,0,0.3,3.53,0.30
Jeff Malott,ID8482408,WPG,LW,246,1,0,0,0,0,3,0,246,0,0,0.3,4.39,0.30


,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,position,date,name,team,opponent,birthdate,id,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins,Unnamed: 0
ID8475167-10-10-2023,1615,1,0,4,0,0,0,0,NaN,0,0,0,1,0,0,1,-1,981,386,248,D,2023-10-10,Victor Hedman,TBL,NSH,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,1.9,0.0,NaN,NaN,NaN,NaN,NaN
ID8475167-14-10-2023,1401,2,1,3,0,1,0,0,NaN,0,0,0,2,0,0,1,0,1191,168,42,D,2023-10-14,Victor Hedman,TBL,DET,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.3,1.0,NaN,NaN,NaN,NaN,NaN
ID8475167-15-10-2023,1328,1,0,2,3,0,0,2,NaN,0,0,0,0,0,0,2,0,1054,132,142,D,2023-10-15,Victor Hedman,TBL,OTT,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.5,0.0,NaN,NaN,NaN,NaN,NaN
ID8475167-17-10-2023,1699,0,0,3,0,0,0,0,NaN,0,0,0,0,0,0,1,1,1237,279,183,D,2023-10-17,Victor Hedman,TBL,BUF,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0.8,0.0,NaN,NaN,NaN,NaN,NaN
ID8475167-19-10-2023,1397,0,0,1,0,0,0,0,NaN,0,0,0,1,0,0,4,-1,1159,165,73,D,2023-10-19,Victor Hedman,TBL,VAN,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,2.1,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47928,1172,1,1,3,3,1,0,0,NaN,0,0,0,0,0,0,3,1,849,123,200,D,2022-04-23,Victor Hedman,TBL,NSH,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.6,1.0,NaN,NaN,NaN,NaN,ID8475167-23-04-2022
48097,1467,4,0,3,2,0,2,0,NaN,0,0,1,0,0,0,0,1,718,427,322,D,2022-04-24,Victor Hedman,TBL,FLA,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,5.5,2.0,NaN,NaN,NaN,NaN,ID8475167-24-04-2022
48704,1419,1,0,5,1,0,1,0,NaN,0,0,0,0,0,0,3,1,1207,120,92,D,2022-04-26,Victor Hedman,TBL,CBJ,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,3.6,1.0,NaN,NaN,NaN,NaN,ID8475167-26-04-2022
49210,1417,2,0,3,0,0,2,2,NaN,0,0,0,1,0,0,1,-1,888,420,109,D,2022-04-28,Victor Hedman,TBL,CBJ,1990-12-18,ID8475167,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,3.8,2.0,NaN,NaN,NaN,NaN,ID8475167-28-04-2022


In [50]:
for index, row in skaters_summary.iterrows():
  last_three_fppg = skaters_summary_last_three.loc[skaters_summary_last_three['id'] == row['id']]['FPPG']
  try:
    skaters_summary.at[index, 'last3FPPG'] = last_three_fppg[0]
  except:
    skaters_summary.at[index, 'last3FPPG'] = 0

In [51]:
goalies_summary_last_three
for index, row in goalies_summary.iterrows():
  last_three_fpp60 = goalies_summary_last_three.loc[goalies_summary_last_three['id'] == row['id']]['FPP60']
  try:
    goalies_summary.at[index, 'last3FPPG'] = last_three_fpp60[0]
  except:
    goalies_summary.at[index, 'last3FPPG'] = 0

In [57]:
skaters_summary_last_three.loc[skaters_summary_last_three.index == 'Tim Stützle']

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,blocked,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,
Tim Stützle,ID8482116,OTT,C,192986,162,63,91,60,425,245,105,150678,35536,6772,366.5,6.84,2.26


In [58]:
goalies_summary.to_csv('goalies.csv')
skaters_summary.to_csv('skaters.csv')